In [24]:
import gym
import gym_super_mario_bros
from gym_super_mario_bros.actions import RIGHT_ONLY, SIMPLE_MOVEMENT, COMPLEX_MOVEMENT
from nes_py.wrappers import JoypadSpace
from IPython.display import clear_output

class GymV26Wrapper(gym.Wrapper):
    def reset(self, **kwargs):
        obs = self.env.reset()
        return obs  # no (obs, info) tuple like gym>=0.26


In [ ]:
env = gym_super_mario_bros.make('SuperMarioBros-1-1-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)

c:\Users\Divya\anaconda3\envs\mario\lib\site-packages\gym\envs\registration.py:593: UserWarning: WARN: The environment SuperMarioBros-1-1-v0 is out of date. You should consider upgrading to version `v3`.
  logger.warn(
c:\Users\Divya\anaconda3\envs\mario\lib\site-packages\gym\core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
c:\Users\Divya\anaconda3\envs\mario\lib\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [ ]:
done=True
for step in range(100000):
    if done: #start the game
        env.reset()
    state, reward, done, info = env.step(env.action_space.sample())
    env.render()
env.close()

In [22]:
from gym.wrappers import FrameStack, GrayScaleObservation
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
from matplotlib import pyplot as plt
from stable_baselines3.common.env_util import make_vec_env

In [28]:
env = gym_super_mario_bros.make('SuperMarioBros-1-1-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)
#Grayscale
env=GrayScaleObservation(env,keep_dim=True)
#Wrap in dummy environment
env=DummyVecEnv([lambda:env])
#stack the frames every 4 frames are stored for ai to have trajectory and memory
env=VecFrameStack(env,4,channels_order='last')


In [29]:
import os
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback

In [30]:
# callback funtion to save model every x steps
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [31]:
CHECKPOINT_DIR='./train/' #location to save model
LOG_DIR='./logs/' #location to save model data

In [32]:
callback= TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)
# used to save model every 10000 steps

In [33]:
model= PPO('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001, n_steps=512)
# cnnpolicy= neural network for image processing
# high lerning rate= bad ai


Using cuda device
Wrapping the env in a VecTransposeImage.


In [35]:
# Train the AI model, this is where the AI model starts to learn
model.learn(total_timesteps=1000000)

TypeError: reset() got an unexpected keyword argument 'seed'

In [ ]:
model.save('thisisatestmodel')